In [1]:
from torch_geometric.datasets import ModelNet
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader

import torch
import torch.nn.functional as F
from torch.nn import Sequential as Seq, Dropout, Linear as Lin, ReLU, BatchNorm1d as BN
from torch_geometric.nn import DynamicEdgeConv, global_max_pool

In [2]:
def MLP(channels, batch_norm=True):
    return Seq(*[
        Seq(Lin(channels[i - 1], channels[i]), ReLU(), BN(channels[i]))
        for i in range(1, len(channels))
    ])

class Net(torch.nn.Module):
    def __init__(self, out_channels, k=20, aggr='max'):
        super().__init__()

        self.conv1 = DynamicEdgeConv(MLP([2 * 3, 64, 64, 64]), k, aggr)
        self.conv2 = DynamicEdgeConv(MLP([2 * 64, 128]), k, aggr)
        self.lin1 = MLP([128 + 64, 1024])

        self.mlp = Seq(
            MLP([1024, 512]), Dropout(0.5), MLP([512, 256]), Dropout(0.5),
            Lin(256, out_channels))

    def forward(self, data):
        pos, batch = data.pos, data.batch
        x1 = self.conv1(pos, batch)
        x2 = self.conv2(x1, batch)
        out = self.lin1(torch.cat([x1, x2], dim=1))
        out = global_max_pool(out, batch)
        out = self.mlp(out)
        return F.log_softmax(out, dim=1)

In [3]:
pre_transform, transform = T.NormalizeScale(), T.SamplePoints(1024)
train_dataset = ModelNet('../data/modelnet10', '10', True, transform, pre_transform)
test_dataset = ModelNet('../data/modelnet10', '10', False, transform, pre_transform)

Extracting ..\data\modelnet10\ModelNet10.zip
Processing...
Done!


In [5]:
train_loader = DataLoader(
    train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(
    test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(train_dataset.num_classes, k=20).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

In [7]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
    return total_loss / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            pred = model(data).max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)


# for epoch in range(1, 201):
#     loss = train()
#     test_acc = test(test_loader)
#     print('Epoch {:03d}, Loss: {:.4f}, Test: {:.4f}'.format(
#         epoch, loss, test_acc))
#     scheduler.step()

KeyboardInterrupt: 

In [8]:
total_loss = 0

In [9]:
for data in train_loader:
    data = data.to(device)
    break
    
optimizer.zero_grad()

In [13]:
print(data)
print(f' points as is: {data.pos.shape}')
print(f' total number of points: {data.batch.shape}')
print(f' number of batches: {data.num_graphs}')

Batch(batch=[32768], pos=[32768, 3], y=[32])
 points as is: torch.Size([32768, 3])
 total number of points: torch.Size([32768])
 number of batches: 32


In [27]:
data.pos[data.batch == 0].shape # select first batch

torch.Size([1024, 3])

In [44]:
data.y

tensor([8, 7, 7, 2, 2, 2, 0, 5, 0, 5, 4, 3, 7, 9, 6, 5, 8, 7, 6, 6, 5, 6, 1, 4,
        8, 2, 7, 1, 9, 5, 7, 0])

In [42]:
out = model(data)

In [43]:
loss = F.nll_loss(out, data.y)

In [ ]:
loss.backward()
total_loss += loss.item() * data.num_graphs
optimizer.step()